# Import Dependencies

In [10]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
import time
from pydub import AudioSegment
from pydub.playback import play

## Global Variables

In [11]:
root = 'ToiletSoundSet'
data_set_folder = root + '/audio'
temp_folder_path = root + '/tmp'
output_folder_path = root + '/sound_data'
validate_output_folder_path = root + '/validate_data'
validation_folder_path = root + '/validation_audio'
target_wav_duration = 4.0
pitch_sounds = [-1, 1, 2, -2]
vary_sounds = [0.93, 1.07]
class_labels = {
    "toilet_flush": 0,
    "shower": 1,
    "door": 2,
    "broken_glass": 3,
    "brushing_teeth": 4,
    "clapping": 5,
    "laughing": 6,
    "door_knock": 7,
    "children_playing": 8
}

## Common Functions

In [12]:
def create_directory_if_not_exist(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

## Sound Augment Functions

In [13]:
def add_slient_sound_if_require(file_path, folder_path, out_dir, new_file_name):
    folder_name = folder_path.split('/')[-1]    
    file_name = file_path.split('/')[-1].split('.')[0]
    
    create_directory_if_not_exist(out_dir)
    
    sound_file_path = file_path
    y, sr = librosa.load(sound_file_path)
    wav_duration = librosa.get_duration(y=y, sr=sr)
    
    if wav_duration < target_wav_duration:
        sound_file_path = add_slient_to_sound(sound_file_path, wav_duration)  
        
    y, sr = librosa.load(sound_file_path, duration=target_wav_duration)
    
    out_file_path = out_dir + '/' + new_file_name + '.wav'
    librosa.output.write_wav(out_file_path, y, sr)
            
    if sound_file_path.find('tmp') > -1:        
        os.remove(sound_file_path)
        
    return out_file_path
    
def stretched_sound(file_path, out_dir, file_name):
    for rate in vary_sounds:
        stretched_wav, sr = stretch_wav(file_path, rate)
        stretched_file_name = file_name + '-vary_' + str(rate) + '.wav'
        librosa.output.write_wav(out_dir + '/' + stretched_file_name, stretched_wav, sr)

def pitch_sound(file_path, out_dir, file_name):
    for pitch_rate in pitch_sounds:
        pitched_wav, sr = shift_wav(file_path, pitch_rate)
        pitched_file_name = file_name + '-pitch_' + str(pitch_rate) + '.wav'
        librosa.output.write_wav(out_dir + '/' + pitched_file_name, pitched_wav, sr)


def add_slient_to_sound(file_path, wav_duration):
    slient_duration = ((target_wav_duration - wav_duration) * 1000) / 2
    if slient_duration < 0:
        return file_path
    left_slient_segment = AudioSegment.silent(duration=slient_duration)
    right_slient_segment = AudioSegment.silent(duration=slient_duration)
    sound = AudioSegment.from_wav(file_path)
    file_name = file_path.split('/')[-1]
    final_sound = left_slient_segment + sound + right_slient_segment
    final_sound_path = temp_folder_path + '/' + file_name
    final_sound.export(final_sound_path, format="wav")
    return final_sound_path

def stretch_wav(wav_file, rate):
    y, sr = librosa.load(wav_file, duration=target_wav_duration)
    stretched_wav = librosa.effects.time_stretch(y, rate=rate)
    return (stretched_wav, sr)

def shift_wav(wav_file, steps):
    y, sr = librosa.load(wav_file, duration=target_wav_duration)
    shift_wav = librosa.effects.pitch_shift(y, sr, n_steps=steps)
    return (shift_wav, sr)

## Augment Training Data

In [14]:
import_folder_paths = []
for root, dirnames, _ in os.walk(data_set_folder):
    for dir_name in dirnames:
        wav_folder_path = os.path.join(root, dir_name)
        import_folder_paths.append(wav_folder_path)
print('import folder path:{}'.format(import_folder_paths))

import folder path:['ToiletSoundSet/audio/toilet_flush', 'ToiletSoundSet/audio/door_knock', 'ToiletSoundSet/audio/children_playing', 'ToiletSoundSet/audio/shower', 'ToiletSoundSet/audio/laughing', 'ToiletSoundSet/audio/clapping', 'ToiletSoundSet/audio/brushing_teeth', 'ToiletSoundSet/audio/broken_glass', 'ToiletSoundSet/audio/door']


In [15]:
imported_counter = 0
for folder_path in import_folder_paths:
    for root, _, files in os.walk(folder_path):
        counter = 0
        folder_name = folder_path.split('/')[-1]
        out_dir = output_folder_path + '/' + folder_name
        
        class_label = class_labels[folder_name]
        if class_label == None:
            print('Label is missing for class:{}'.format(folder_name))
            continue
        
        print('Import data for folder:{}'.format(folder_path))
        
        for file in files:
            file_name, file_extension = os.path.splitext(file)
            if file_extension != '.wav': continue
            if imported_counter % 500 == 0:
                print('Import in progress: {}'.format(imported_counter))
            file_path = os.path.join(root, file)
            new_file_name = str(counter).zfill(4) + '-' + str(class_label)
            
            # Data Augmentation
            file_path = add_slient_sound_if_require(file_path, folder_path, out_dir, new_file_name)
            stretched_sound(file_path, out_dir, new_file_name)
            pitch_sound(file_path, out_dir, new_file_name)

            imported_counter += 1
            counter += 1
print('Finish')

Import data for folder:ToiletSoundSet/audio/toilet_flush
Import in progress: 0
Import data for folder:ToiletSoundSet/audio/door_knock
Import data for folder:ToiletSoundSet/audio/children_playing
Import data for folder:ToiletSoundSet/audio/shower
Import data for folder:ToiletSoundSet/audio/laughing
Import data for folder:ToiletSoundSet/audio/clapping
Import data for folder:ToiletSoundSet/audio/brushing_teeth
Import data for folder:ToiletSoundSet/audio/broken_glass
Import data for folder:ToiletSoundSet/audio/door
Finish


## Augment Validation Data

In [16]:
validate_set_folder_paths = []
for root, dirnames, _ in os.walk(validation_folder_path):
    for dir_name in dirnames:        
        wav_folder_path = os.path.join(root, dir_name)
        validate_set_folder_paths.append(wav_folder_path)
print('import folder path:{}'.format(validate_set_folder_paths))

import folder path:['ToiletSoundSet/validation_audio/toilet_flush', 'ToiletSoundSet/validation_audio/door_knock', 'ToiletSoundSet/validation_audio/children_playing', 'ToiletSoundSet/validation_audio/shower', 'ToiletSoundSet/validation_audio/laughing', 'ToiletSoundSet/validation_audio/clapping', 'ToiletSoundSet/validation_audio/brushing_teeth', 'ToiletSoundSet/validation_audio/broken_glass', 'ToiletSoundSet/validation_audio/door']


In [18]:
for folder_path in validate_set_folder_paths:
    for root, _, files in os.walk(folder_path):
        counter = 0
        folder_name = folder_path.split('/')[-1]
        out_dir = validate_output_folder_path + '/' + folder_name
        
        class_label = class_labels[folder_name]
        if class_label == None:
            print('Label is missing for class:{}'.format(folder_name))
            continue
        print('Import data for folder:{}'.format(folder_name))
        for file in files:
            file_name, file_extension = os.path.splitext(file)
            if file_extension != '.wav': continue
            file_path = os.path.join(root, file)            
            new_file_name = str(counter).zfill(4) + '-' + str(class_label)            
            add_slient_sound_if_require(file_path, folder_path, out_dir, new_file_name)
            counter += 1
print('Finish')